In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# -------------------------------
# Step 1: Combine Energy Consumption Files
# -------------------------------
for num in range(0, 112):
    df = pd.read_csv(r"C:\Users\karim\Desktop\New folder\daily_dataset\block_" + str(num) + ".csv")
    df = df[['day', 'LCLid', 'energy_sum']]  # Keep only relevant columns
    df.reset_index()  # Reset index
    df.to_csv(r"C:\Users\karim\Desktop\New folder\output\hc_" + str(num) + ".csv")  # Save cleaned files

# Combine all individual files into a single dataset
fout = open(r"C:\Users\karim\Desktop\New folder\energy.csv", "a")
for line in open(r"C:\Users\karim\Desktop\New folder\output\hc_0.csv"):
    fout.write(line)
for num in range(0, 112):
    f = open(r"C:\Users\karim\Desktop\New folder\output\hc_" + str(num) + ".csv")
    f.readline()  # Skip header for subsequent files
    for line in f:
        fout.write(line)
    f.close()
fout.close()

# -------------------------------
# Step 2: Load and Process Energy Data
# -------------------------------
energy = pd.read_csv(r"C:\Users\karim\Desktop\New folder\energy.csv")
housecount = energy.groupby('day')[['LCLid']].nunique()  # Count unique households per day
energy = energy.groupby('day')[['energy_sum']].sum()  # Sum energy consumption per day
energy = energy.merge(housecount, on='day')  # Add household count
energy = energy.reset_index()  # Reset index

# Add average energy consumption per household
energy['avg_energy'] = energy['energy_sum'] / energy['LCLid']

# Convert 'day' column to datetime
energy['day'] = pd.to_datetime(energy['day'], format='%Y-%m-%d').dt.date

# Print data range
print("Starting Point of Data at Day Level:", min(energy['day']))
print("Ending Point of Data at Day Level:", max(energy['day']))

# -------------------------------
# Step 3: Load and Process Weather Data
# -------------------------------
weather = pd.read_csv(r"C:\Users\karim\Desktop\New folder\weather_daily_darksky.csv")
weather['day'] = pd.to_datetime(weather['time']).dt.date  # Convert timestamp to date
weather = weather.dropna()  # Drop rows with missing values

# Keep relevant columns
weather = weather[['temperatureMax', 'humidity', 'windSpeed', 'cloudCover', 'visibility', 'day']]

# Merge energy and weather data
weather_energy = energy.merge(weather, on='day')

# -------------------------------
# Step 4: Visualize Relationships
# -------------------------------
fig, ax1 = plt.subplots(figsize=(20, 5))
ax1.plot(weather_energy['day'], weather_energy['temperatureMax'], color='tab:orange', label='Temperature Max')
ax1.plot(weather_energy['day'], weather_energy['avg_energy'], color='tab:blue', label='Average Energy')
ax1.set_ylabel('Temperature/Energy')
plt.legend()
plt.title('Energy Consumption and Temperature')
plt.show()

# -------------------------------
# Step 5: Scaling & Clustering for Weather Conditions
# -------------------------------
scaler = MinMaxScaler()
weather_scaled = scaler.fit_transform(weather_energy[['temperatureMax', 'humidity', 'windSpeed']])

# Elbow curve to determine optimal clusters
Nc = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in Nc]
scores = [kmeans[i].fit(weather_scaled).score(weather_scaled) for i in range(len(kmeans))]
plt.plot(Nc, scores)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

# Fit KMeans with 3 clusters
kmeans = KMeans(n_clusters=3, max_iter=600)
kmeans.fit(weather_scaled)
weather_energy['weather_cluster'] = kmeans.labels_

# -------------------------------
# Step 6: Add Holiday Indicator
# -------------------------------
holiday = pd.read_csv(r"C:\Users\karim\Desktop\New folder\uk_bank_holidays.csv")
holiday['Bank holidays'] = pd.to_datetime(holiday['Bank holidays'], format='%Y-%m-%d').dt.date
weather_energy = weather_energy.merge(holiday, left_on='day', right_on='Bank holidays', how='left')
weather_energy['holiday_ind'] = np.where(weather_energy['Bank holidays'].isna(), 0, 1)

# -------------------------------
# Step 7: Train-Test Split
# -------------------------------
weather_energy.set_index(['day'], inplace=True)
model_data = weather_energy[['avg_energy', 'weather_cluster', 'holiday_ind']]

# 70-30 train-test split
train = model_data.iloc[0:(len(model_data) - 30)]
test = model_data.iloc[len(train):]

# -------------------------------
# Step 8: Test for Stationarity
# -------------------------------
t = sm.tsa.adfuller(train['avg_energy'], autolag='AIC')
print(pd.Series(t[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used']))

# -------------------------------
# Step 9: Build and Evaluate SARIMAX Model
# -------------------------------
endog = train['avg_energy']
exog = train[['weather_cluster', 'holiday_ind']]
mod = SARIMAX(endog=endog, exog=exog, order=(7, 1, 1), seasonal_order=(1, 1, 0, 12), trend='c')
model_fit = mod.fit()

# Predictions
predict = model_fit.predict(start=len(train), end=len(train)+len(test)-1, exog=test[['weather_cluster', 'holiday_ind']])
test['predicted'] = predict.values

# Calculate Errors
test['residual'] = abs(test['avg_energy'] - test['predicted'])
MAE = test['residual'].mean()
MAPE = (test['residual'] / test['avg_energy']).mean() * 100
print("MAE:", MAE)
print("MAPE:", MAPE)
